In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 자기효능감

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9E%90%EA%B8%B0%ED%9A%A8%EB%8A%A5%EA%B0%90&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9E%90%EA%B8%B0%ED%9A%A8%EB%8A%A5%EA%B0%90

In [4]:
paper_link_list = []
#21
for i in range(21):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9E%90%EA%B8%B0%ED%9A%A8%EB%8A%A5%EA%B0%90&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9E%90%EA%B8%B0%ED%9A%A8%EB%8A%A5%EA%B0%90"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

간호사의 감성지능이 자기효능감과 직무스트레스에 미치는 영향 : 자기효능감의 매개효과 검증
영아 보육교사의 상해예방을 위한 감독과 자기 효능감 및 직무 스트레스의 관계 연구
대학생의 진로준비행동과 진로불확실성, 자기개념명확성, 진로결정자기효능감, 진로통찰력 간의 구조적 관계
미술치료사의 자의식과 자기효능감에 관한 연구
만성 폐쇄성 폐질환 환자의 증상경험, 자기효능감, 우울과 약물복용 이행 간의 관계
창의적 문제해결(CPS)모형을 적용한 집단미술치료가 미혼모의 자기효능감과 문제해결력에 미치는 효과
항공 응용 분야 : 서비스교육이 자기효능감과 서비스 지향성에 미치는 영향에 관한 연구 -항공서비스학과 학생을 대상으로-
대기업 사무직 근로자의 비판적 성찰 업무 행동과 의사결정 참여, 리더-구성원 교환 관계, 자기효능감 및 변화 준비성의 인과적 관계
원예치료가 시설아동의 자기효능감 및 정서에 미치는 영향
전문대학 학사학위 전공심화과정 학습자의 학습동기와 등록 유인가, 자기효능감, 목표지향성 및 학업성패 귀인성향의 관계
10
게슈탈트 집단미술치료가 지역아동센터 아동의 자기효능감과 또래관계에 미치는 효과
원저 : 구강보건 보조 인력의 자기 효능감과 직무만족도의 관계연구
뉴 실버세대의 자기효능감과 외모관심이 외모관리행동과 의복구매 행동에 미치는 영향
남성노인의 일상생활 수행능력과 자기효능감 및 양생행위
대학생의 진로결정자기효능감과 셀프리더십, 개인이 인식한 문제해결력 및 진로동기의 인과적 관계
작업능력강화프로그램이 산업재해근로자의 원 직무복귀에 미치는 효과 : 자기효능감, 우울, 재활동기를 중심으로
시각적·청각적 범주화 훈련을 통한 뇌졸중 환자의인지 향상 및 자기효능감 관련 연구
즉흥연주기법을 중심으로 한 음악치료가 지적장애근로자의 자기효능감 증진에 미치는 효과
Perceived Efficacy and Goal Setting System과 자기효능감 검사의 상관관계 연구
푸드아트테라피가 청각장애청소년의 자기효능감과 사회성에 미치는 효과
20
건강관리사의 지식, 태도, 기술, 

결절교체 유방모형을 이용한 인지,행동 통합 유방암 예방교육 효과 -간호학생을 대상으로-
남자대학생의 성 경험 유무에 따라 콘돔사용 의도에 영향을 미치는 예측용인
방송인의 뷰티서비스 관계 연구
인지부하이론을 적용한 약물계산훈련용 스마트폰 어플리케이션의 개발 및 효과
노인의 양생(養生)행위에 영향을 미치는 요인
결혼 이주 여성의 삶의 질에 영향을 미치는 요인
응급의료종사자의 소진 영향요인 : 부산, 경남 지역 119구급대원 중심으로
대학생 흡연자의 금연 준비단계 관련 요인
기업체 종사자의 변화에 대한 준비성과 관련 변인의 인과분석
여성노인의 양생행위 경로모형 분석
200
범이론 모형을 적용한 고혈압 환자 약물순응 중재 프로그램 개발 및 적용 효과
다수준 건강증진 모형에 기반 한 건강증진 프로그램 효과
202


In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b6c9b5acdb292943ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f410ec672b8afa03ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6f7a8aab720359a4b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=dbf82c5e7c727285ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=583696eb6a824e13b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=3cb73bbeeadedd8effe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b3478944afad94d3ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ab44a593fbafe3cdffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0fce695311afa66d4884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

202

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 202/202 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b6c9b5acdb292943ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f410ec672b8afa03ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6f7a8aab720359a4b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=dbf82c5e7c727285ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=583696eb6a824e13b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=3cb73bbeeadedd8effe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b3478944afad94d3ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ab44a593fbafe3cdffe0bdc3ef48d419',
 'http://www.ris

In [9]:
len(paper_url)

202

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./5_자기효능감.xlsx')
    
print("Running Time : ", time.time() - start)

0 ERROR
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 ERROR
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 ERROR
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 ERROR
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 ERROR
59 paper clear
60 paper clear
61 ERROR
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 paper clear
68 paper clear
69 paper cl